<a href="https://colab.research.google.com/github/Pa-hm/projeto-ep3-mac/blob/main/ep3_mac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FileNotFoundError: inventario.csv not found.

In [ ]:
import numpy as np

def le_dados():
    inventario = np.genfromtxt("inventario.csv", dtype=np.int32,
                               skip_header=1, delimiter=",")
    operacoes = np.genfromtxt("operacoes.csv", dtype=np.int32,
                              skip_header=1, delimiter=",")
    clientes = np.genfromtxt("clientes.csv", dtype='U40',
                             skip_header=1, delimiter=",")
    lojas = np.genfromtxt("lojas.csv", dtype='U40',
                          skip_header=1, delimiter=",")
    produtos = np.genfromtxt("produtos.csv", dtype='U40',
                             skip_header=1, delimiter=",")
    return inventario, operacoes, clientes, lojas, produtos

def preco_item(inventario, id_loja, id_produto):
    for linha in inventario:
        if int(linha[0]) == int(id_loja) and int(linha[1]) == int(id_produto):
            return int(linha[2])
    pass

def nome_loja(lojas, id_loja):
    for linha in lojas:
        if int(linha[0]) == int(id_loja):
            return linha[1]
    pass

def nome_produto(produtos, id_produto):
    for linha in produtos:
        if int(linha[0]) == int(id_produto):
            return linha[1]
    pass

def nome_cliente(clientes, id_cliente):
    for linha in clientes:
        if int(linha[0]) == int(id_cliente):
            return linha[1]
    pass


def remove_estoque(inventario, id_loja, id_produto, quantidade):
   for linha in inventario:
        if linha[0] == id_loja and linha[1] == id_produto:
            linha[3] -= quantidade




def soma_caixa(lojas, id_loja, valor):
     for linha in lojas:
        if int(linha[0]) == int(id_loja):
           linha[2] = str(valor+int(linha[2]))


def atualiza_estoques(inventario, operacoes):
   for linha in operacoes:
      remove_estoque(inventario,linha[1],linha[2],linha[3])
   np.savetxt("novo_inventario.csv",inventario,fmt="%d",delimiter=",",header="id_loja,id_produto,quantidade")


def atualiza_caixas(inventario, lojas, operacoes):
    for linha in operacoes:
        id_loja = linha[1]
        id_produto = linha[2]
        quantidade = linha[3]
        valor = preco_item(inventario, id_loja, id_produto) * quantidade
        soma_caixa(lojas,id_loja,valor)
    np.savetxt("novo_lojas.csv", lojas, "%s", delimiter=",",header="id_loja,nome_loja,caixa", comments="")

def vendas_por_produto(operacoes, produtos):
    nprodutos = produtos.shape[0]
    vendas = np.zeros([nprodutos, 2], dtype=np.int32)
    for i in range(nprodutos):
        vendas[i,0] = produtos[i,0]
    nops = operacoes.shape[0]
    for i in range(nops):
        id_cliente, id_loja, id_produto, quantidade = operacoes[i]
        for j in range(nprodutos):
            if vendas[j,0] == id_produto:
                vendas[j,1] += quantidade
    return vendas

def quantidade_vendidos(vendas, id_produto):
    for linha in vendas:
        if linha[0] == id_produto:
            return linha[1]
    pass

def reposicao(inventario, operacoes, produtos):
    n_inventario = inventario.shape[0]
    reposicao = np.zeros([n_inventario, 3], dtype=np.int32)
    for i in range(n_inventario):
      reposicao[i,0]= inventario[i,0]
      reposicao[i,1]= inventario[i,1]
      if inventario[i,3] > 0:
         reposicao[i,2] = 0
      else:
         vendas = vendas_por_produto(operacoes, produtos)
         reposicao[i,2] = quantidade_vendidos(vendas,inventario[i,1])//2 +1
    np.savetxt("reposicao_estoque.csv", reposicao, "%d", delimiter=",",header="id_loja,id_produto,quantidade", comments="")
    return reposicao




def faturamento_por_loja(inventario, operacoes, lojas):
    nop= operacoes.shape[0]
    nlojas = lojas.shape[0]
    faturamento = np.zeros([nlojas,2], dtype=np.int32)
    for i in range(nlojas):
        faturamento[i,0] = lojas[i,0]
        faturamento[i,1] = 0
        for j in range(nop):
            if int(lojas[i,0]) == int(operacoes[j,1]):
              faturamento[i,1] += operacoes[j,3] * preco_item(inventario,int(lojas[i,0]),operacoes[j,2])
    return faturamento
    pass

def relatorio_lojas(inventario, operacoes, lojas):
    faturamentos = faturamento_por_loja(inventario, operacoes, lojas)
    nlojas = lojas.shape[0]
    for i in range(nlojas):
        id_loja = faturamentos[i,0]
        valor = faturamentos[i,1]
        loja = nome_loja(lojas, id_loja)
        if valor > 0:
            print(f'A loja "{loja}" faturou R${valor}')
        else:
            print(f'A loja "{loja}" não teve vendas')

def gasto_por_cliente(inventario, operacoes, clientes):
    nop= operacoes.shape[0]
    nclientes = clientes.shape[0]
    gastos = np.zeros([nclientes,2], dtype=np.int32)
    for i in range(nclientes):
        gastos[i,0] = clientes[i,0]
        gastos[i,1] = 0
        for j in range(nop):
            if int(clientes[i,0]) == int(operacoes[j,0]):
              gastos[i,1] += operacoes[j,3] * preco_item(inventario,int(operacoes[j,1]),operacoes[j,2])
    return gastos

def relatorios_individuais(inventario, operacoes, clientes, lojas):
    gastos = gasto_por_cliente(inventario, operacoes, clientes)
    nclientes = gastos.shape[0]
    clientes_ativos = []
    faturamento = 0
    for i in range(nclientes):
        if gastos[i,1] > 0:
            id_cliente = gastos[i,0]
            gasto = gastos[i,1]
            cliente = nome_cliente(clientes, id_cliente)
            faturamento += gasto
            clientes_ativos.append(cliente)
            print(f"{cliente} gastou no total R${gasto}")
    return faturamento, clientes_ativos

def relatorio_clientes(inventario, operacoes, clientes, lojas):
    faturamento, clientes_ativos = relatorios_individuais(inventario,
                                                          operacoes,
                                                          clientes, lojas)

    if len(clientes_ativos) == 0:
        print("Nenhum cliente fez compras no shopping hoje!")
        return

    nomes = ', '.join(clientes_ativos)
    ultima_virgula = nomes.rfind(", ")
    nomes = (nomes[:ultima_virgula] + " e " + nomes[ultima_virgula +2:])
    print(f"{len(clientes_ativos)} clientes fizeram compras no shopping:",
          nomes)
    media = faturamento / len(clientes_ativos)
    print(f"Os clientes gastaram em média R${media:.2f}")
    print(f"O faturamento total do shopping foi R${faturamento}")

def relatorio_estoques(reposicoes, lojas, produtos):
    nitems = reposicoes.shape[0]
    for i in range(nitems):
        id_loja, id_produto, quantidade = reposicoes[i]
        if quantidade > 0:
            loja = nome_loja(lojas, id_loja)
            produto = nome_produto(produtos, id_produto)
            print(f'A loja "{loja}" deve comprar {quantidade} '
                  f'unidades do produto {produto}')
def main():
    inventario, operacoes, clientes, lojas, produtos = le_dados()
    atualiza_estoques(inventario, operacoes)
    atualiza_caixas(inventario, lojas, operacoes)
    reposicoes = reposicao(inventario, operacoes, produtos)
    print("==== Relatório do dia ====")
    print()
    relatorio_clientes(inventario, operacoes, clientes, lojas)
    relatorio_lojas(inventario, operacoes, lojas)
    relatorio_estoques(reposicoes, lojas, produtos)

if __name__ == "__main__":
    main()